In [1]:
import numpy as np
import pandas as pd
import copy
import pickle
import random

In [2]:
def preprocess_text(text):
    '''получаем из текста список слов'''
    text = text.str.lower()
    #в регулярном выражении \d означает "любая цифра" \W - не цифра и не буква
    #и + означает "один или более text = text.str.replace('\d+|\W+',' ')
    text = text.str.replace('_',' ')
    #text = text.str.replace('@',' абазин ')
    #text = text.str.replace('\B,','_')
    text = text.str.replace('\d+|\W+',' ')
    text = text.str.replace('_',' ')
    text = text.str.replace('ё','е')
    text = text.str.strip()
    text = text.str.split()   
    return text

In [3]:
# загрузка файла
path_to_folder = r"C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы"
file_to_load = "HeadHunter_test_dummy.csv"
path = path_to_folder + "//" + file_to_load

file_df = pd.read_csv(path)

In [4]:
file_df

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,rest_recovery_rating
0,3,Москва,Специалист по строительному контролю систем за...,"рассуждений всплывал из-за первых, что он в од...","тебя белые ресницы единственно оттого только, ...",5,5,5,5,5,5
1,4,Москва,Редактор,"все, и гостья и графиня, оглянулись на нее, ка...",я вот здесь буду ждать...\n Свидригайлов с...,4,5,4,5,4,4
2,13,Тверь,Старший штурман авиационного училища,"Что прикажете, графинюшка?\n - Вот что, мо...","- Вечером, Родя, - отвечала Пульхерия Алек...",2,5,5,5,5,5
3,14,Уфа,Инспектор центра занятости населения,"Анне Павловне, почему он полагал, что план абб...",же новых они в то же время весьма часто не зам...,2,5,5,2,4,4
4,17,Москва,Артист вокально-инструментального ансамбля,с разных сторон равномерно и не умолкая шумели...,"- Необходимо отправиться по делу, и таким о...",5,5,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...
50646,192819,Санкт-Петербург,Технолог по выращиванию рыбы,"- Ах, это совершенно всё равно, - быстро за...","несколько сбившись, - вещи мои стоят всего пят...",3,4,3,4,3,1
50647,192821,Москва,Тренер - оператор видеозаписи сборной команды,"от быстроты вертящиеся блоки, а соседнее колес...","выкупленные и невыкупленные, - браслеты, цепоч...",3,5,5,2,4,4
50648,192830,Алматы,Специалист по противопожарной профилактике,"как будто он говорил: ""Вы имеете полное право ...",г. в Москве происходил военно-полевой суд над ...,2,2,1,1,4,1
50649,192832,Краснодар,Специалист по платежным услугам (транзакционно...,"Выехав за деревню, беспрестанно обгоняя и вс...",совесть в высшей степени спокойна на этот счет...,1,5,2,5,4,3


In [5]:
# заменяем NA на названия столбцов
for key in file_df.keys()[1:5]:
    file_df[key].fillna(value=key, inplace=True)    

In [6]:
# препроцессинг: токенизация
file_df['city'] = preprocess_text(file_df['city'])
file_df['position'] = preprocess_text(file_df['position'])
file_df['positive'] = preprocess_text(file_df['positive'])
file_df['negative'] = preprocess_text(file_df['negative'])

<ipython-input-2-df24433e5016>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  text = text.str.replace('\d+|\W+',' ')


In [7]:
# заполняем пустые списки, появившиеся после обработки
for l in file_df['negative']:
    if len(l) == 0:
        l.append('negative')
for l in file_df['positive']:
    if len(l) == 0:
        l.append('positive')
s = 0
for l in file_df['negative']:
    if len(l) == 0:
        s += 1
for l in file_df['positive']:
    if len(l) == 0:
        s += 1
s

0

In [8]:
# загружаем словарь для перевода слов русского языка в основную форму
path_to_dict_rus = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы\words_relation_for_lema.csv'
dict_rus = pd.read_csv(path_to_dict_rus)
dict_rus['word'] = dict_rus['word'].str.replace('ё','е')
dict_rus['word_stem'] = dict_rus['word_stem'].str.replace('ё','е')
#dict_rus.sort_values(by=['word'])
dict_rus.set_index('word', inplace=True)
dict_rus.sort_index(inplace=True)

In [9]:
dict_rus[85:]

,word_stem
word,
адъютантов,адъютант
адъютантом,адъютант
адъютанту,адъютант
адъютанты,адъютант
аж,аж
...,...
ящика,ящик
ящиками,ящик
ящике,ящик


In [10]:
# переводим слова в основные формы, добавляем два новых столбца в file_df
# path_to_load_words = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\en_ru_dic_lmmatized.csv'
# en_rus_dic = pd.read_csv(path_to_load_words, index_col=0)

stop_words = ['без', 'более', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 
              'ведь', 'весь', 'вдоль', 'вместо', 'вне', 'вниз', 'внизу', 'внутри', 'во', 
              'вокруг', 'вот', 'все', 'всегда', 'всего', 'всех', 'вы', 'где', 'да', 'давай', 
              'давать', 'даже', 'для', 'до', 'достаточно', 'его', 'ее', 'ее', 'если', 'есть', 
              'еще', 'же', 'за', 'здесь', 'из', 'и', 'или', 'им', 'иметь', 
              'их', 'как', 'то', 'кто', 'когда', 'кроме', 'кто', 'ли', 'либо', 'мне', 'может', 
              'мои', 'мой', 'мы', 'на', 'навсегда', 'над', 'надо', 'наш', 'не', 'него', 'нее', 'нет', 
              'ни', 'них', 'но', 'ну', 'об', 'однако', 'он', 'она', 'они', 'оно', 'от', 'отчего', 'очень', 
              'по', 'под', 'после', 'потому', 'что', 'почти', 'при', 'про', 'снова', 'со', 'так', 
              'также', 'такие', 'такой', 'там', 'те', 'тем', 'то', 'того', 'тоже', 'той', 'только', 'том', 
              'тут', 'ты', 'уже', 'хотя', 'чего', 'чей', 'чем', 'что', 'чтобы', 'чье', 'чья', 'эта', 'эти', 'это', 
              "а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто",
              "буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная",
              "важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали",
              "вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видеть","вместе",
              "вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать",
              "восемь","восьмой","вот","впрочем","времени","время","все","всегда","всего","всем","всеми",
              "всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный",
              "глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да",
              "давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый",
              "двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать",
              "девятый","девять","действительно","дел","делать","дело","день","деньги","десятый","десять",
              "для","до","довольно","долго","должно","должный","дом","дорога","друг","другая","другие",
              "других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если",
              "есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за",
              "занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать",
              "значит","значить","и","идти","из","или","им","именно","иметь","ими","имя","иногда",
              "их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой",
              "кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого",
              "которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли",
              "лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля",
              "менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много",
              "многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог",
              "могут","мож","может","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы",
              "на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ",
              "нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко",
              "нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко",
              "несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда",
              "никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь",
              "ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать",
              "однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно",
              "опять","особенно","остаться","от","ответить","отец","отовсюду","отсюда","очень","первый",
              "перед","писать","плечо","по","под","подумать","пожалуйста","позже","пойти","пока",
              "пол","получить","помнить","понимать","понять","пор","пора","после","последний",
              "посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при",
              "про","просто","против","процентов","пятнадцатый","пятнадцать","пятый","пять",
              "работа","работать","раз","разве","рано","раньше","ребенок","решить","россия",
              "рука","русский","ряд","рядом","с","сам","сама","сами","самим","самими","самих",
              "само","самого","самой","самом","самому","саму","самый","свет","свое","своего",
              "своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня",
              "седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал",
              "сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова",
              "со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый",
              "стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая",
              "также","таки","такие","такое","такой","там","твой","твоя","твоё","те","тебе","тебя",
              "тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только",
              "том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты",
              "тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотеть","хоть",
              "хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через",
              "четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть",
              "шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих",
              "это","этого","этой","этом","этому","этот","эту","я"] 

list_negative = copy.deepcopy(list(file_df['negative']))
list_positive = copy.deepcopy(list(file_df['positive']))

for lis in list_negative:
    for i, word in enumerate(lis):
        if word in dict_rus.index:
            if word not in stop_words:
                lis[i] = str(dict_rus.loc[word].values[0]) + 'neg'
        else:
            lis[i] = '' #lis[i] = 'йцук' + 'neg'
    lis = list(set(lis)) # удаляем повторения слов в тексте

for lis in list_positive:
    for i, word in enumerate(lis):
        if (word in dict_rus.index and word not in stop_words):
            if word not in stop_words:
                lis[i] = str(dict_rus.loc[word].values[0]) + 'pos'
        else:
            lis[i] = '' #lis[i] = 'йцук' + 'pos'
    lis = list(set(lis)) # удаляем повторения слов в тексте

file_df['positive_1'] = list_positive
file_df['negative_1'] = list_negative

In [11]:
# добавляем к числовым признакам названия столбцов
k = list(file_df.keys()[5:11])
for ind in k:
    i = ind.split('_')[0]
    file_df[ind] = file_df[ind].apply(lambda x: (i + str(x)).split())


In [12]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1', 'negative_1'],
      dtype='object')

In [13]:
lis = []
for i, key_1 in enumerate(file_df.keys()[5:11]):    
    for key_2 in file_df.keys()[(5 + i + 1):11]:
        lis1 = [[] for xc in range(len(file_df))]
        for j in range(len(file_df)):
            w1 = file_df[key_1].iloc[j]
            w2 = file_df[key_2].iloc[j]
            lis1[j].append(w1[0] + w2[0])
        lis.append(lis1)
for jj in range(len(lis)):
    file_df[str(jj)] = lis[jj]

In [14]:
file_df

,review_id,city,position,positive,negative,salary_rating,team_rating,managment_rating,career_rating,workplace_rating,...,5,6,7,8,9,10,11,12,13,14
0,3,[москва],"[специалист, по, строительному, контролю, сист...","[рассуждений, всплывал, из, за, первых, что, о...","[тебя, белые, ресницы, единственно, оттого, то...",[salary5],[team5],[managment5],[career5],[workplace5],...,[team5managment5],[team5career5],[team5workplace5],[team5rest5],[managment5career5],[managment5workplace5],[managment5rest5],[career5workplace5],[career5rest5],[workplace5rest5]
1,4,[москва],[редактор],"[все, и, гостья, и, графиня, оглянулись, на, н...","[я, вот, здесь, буду, ждать, свидригайлов, сел...",[salary4],[team5],[managment4],[career5],[workplace4],...,[team5managment4],[team5career5],[team5workplace4],[team5rest4],[managment4career5],[managment4workplace4],[managment4rest4],[career5workplace4],[career5rest4],[workplace4rest4]
2,13,[тверь],"[старший, штурман, авиационного, училища]","[что, прикажете, графинюшка, вот, что, мой, др...","[вечером, родя, отвечала, пульхерия, александр...",[salary2],[team5],[managment5],[career5],[workplace5],...,[team5managment5],[team5career5],[team5workplace5],[team5rest5],[managment5career5],[managment5workplace5],[managment5rest5],[career5workplace5],[career5rest5],[workplace5rest5]
3,14,[уфа],"[инспектор, центра, занятости, населения]","[анне, павловне, почему, он, полагал, что, пла...","[же, новых, они, в, то, же, время, весьма, час...",[salary2],[team5],[managment5],[career2],[workplace4],...,[team5managment5],[team5career2],[team5workplace4],[team5rest4],[managment5career2],[managment5workplace4],[managment5rest4],[career2workplace4],[career2rest4],[workplace4rest4]
4,17,[москва],"[артист, вокально, инструментального, ансамбля]","[с, разных, сторон, равномерно, и, не, умолкая...","[необходимо, отправиться, по, делу, и, таким, ...",[salary5],[team5],[managment5],[career5],[workplace5],...,[team5managment5],[team5career5],[team5workplace5],[team5rest5],[managment5career5],[managment5workplace5],[managment5rest5],[career5workplace5],[career5rest5],[workplace5rest5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50646,192819,"[санкт, петербург]","[технолог, по, выращиванию, рыбы]","[ах, это, совершенно, все, равно, быстро, заго...","[несколько, сбившись, вещи, мои, стоят, всего,...",[salary3],[team4],[managment3],[career4],[workplace3],...,[team4managment3],[team4career4],[team4workplace3],[team4rest1],[managment3career4],[managment3workplace3],[managment3rest1],[career4workplace3],[career4rest1],[workplace3rest1]
50647,192821,[москва],"[тренер, оператор, видеозаписи, сборной, команды]","[от, быстроты, вертящиеся, блоки, а, соседнее,...","[выкупленные, и, невыкупленные, браслеты, цепо...",[salary3],[team5],[managment5],[career2],[workplace4],...,[team5managment5],[team5career2],[team5workplace4],[team5rest4],[managment5career2],[managment5workplace4],[managment5rest4],[career2workplace4],[career2rest4],[workplace4rest4]
50648,192830,[алматы],"[специалист, по, противопожарной, профилактике]","[как, будто, он, говорил, вы, имеете, полное, ...","[г, в, москве, происходил, военно, полевой, су...",[salary2],[team2],[managment1],[career1],[workplace4],...,[team2managment1],[team2career1],[team2workplace4],[team2rest1],[managment1career1],[managment1workplace4],[managment1rest1],[career1workplace4],[career1rest1],[workplace4rest1]
50649,192832,[краснодар],"[специалист, по, платежным, услугам, транзакци...","[выехав, за, деревню, беспрестанно, обгоняя, и...","[совесть, в, высшей, степени, спокойна, на, эт...",[salary1],[team5],[managment2],[career5],[workplace4],...,[team5managment2],[team5career5],[team5workplace4],[team5rest3],[managment2career5],[managment2workplace4],[managment2rest3],[career5workplace4],[career5rest3],[workplace4rest3]


In [15]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1', 'negative_1',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'],
      dtype='object')

In [16]:
# Все поля соединяем в один текст
k = ['city', 'position', 'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1',
       'negative_1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14']

corpus = [[] for x in range(file_df.shape[0])]
for ind in k:
    for i, x in enumerate(file_df[ind]):
        corpus[i] = corpus[i] + file_df[ind].iloc[i]
corpus = list(corpus)
corpus_1 = []
for text in corpus:
    string = []
    text = list(text)
    for word in text:
        if type(word) == type('text'):
            string.append(word)
    corpus_1.append(' '.join(string))

In [17]:
file_df['text'] = corpus_1

In [18]:
file_df['text'].isna().sum()

0

In [19]:
file_df.keys()

Index(['review_id', 'city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1', 'negative_1',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', 'text'],
      dtype='object')

In [20]:
# удаляем ненужные столбцы из файла для тренировки модели
file_df.drop(columns=['city', 'position', 'positive', 'negative',
       'salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'positive_1',
       'negative_1', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14'], inplace=True)
file_df

,review_id,text
0,3,москва специалист по строительному контролю си...
1,4,москва редактор salary4 team5 managment4 caree...
2,13,тверь старший штурман авиационного училища sal...
3,14,уфа инспектор центра занятости населения salar...
4,17,москва артист вокально инструментального ансам...
...,...,...
50646,192819,санкт петербург технолог по выращиванию рыбы s...
50647,192821,москва тренер оператор видеозаписи сборной ком...
50648,192830,алматы специалист по противопожарной профилакт...
50649,192832,краснодар специалист по платежным услугам тран...


In [21]:
file_df['text'][0]

'москва специалист по строительному контролю систем защиты от коррозии salary5 team5 managment5 career5 workplace5 rest5 рассуждениеpos всплыватьpos   первоеpos    одинpos     думатьpos   ничтожествоpos  мечтатьpos тебя белыйneg ресницаneg единственноneg оттогоneg только что в  великийneg тридцатьneg пять саженьneg высотаneg и вывестиneg ясныйneg точноneg прогрессивныйneg и даже с либеральныйneg оттенокneg братьсяneg ну хочешь паряneg salary5team5 salary5managment5 salary5career5 salary5workplace5 salary5rest5 team5managment5 team5career5 team5workplace5 team5rest5 managment5career5 managment5workplace5 managment5rest5 career5workplace5 career5rest5 workplace5rest5'

In [22]:
# записываем файл, готовый для тестирования модели
path_to_save = r'C:\Users\Dima\Desktop\СберУниверситет\Дипломный проект\Диплом файлы\file_df_set_test_MLP_new.csv'
file_df.to_csv(path_to_save)